In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible

In [ ]:
import json
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [ ]:
notebook_login()


In [ ]:
#model_id = "google/gemma-7b-it"
model_id = "google/codegemma-7b-it"


In [ ]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it", add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json

# File paths
input_file_path = '/content/drive/MyDrive/ACL/MultilingualLLMBias/GPT3.5-finetune-data/stacked_combined.jsonl'
output_file_path = '/content/drive/MyDrive/ACL/MultilingualLLMBias/GPT3.5-finetune-data/formatted_stacked_combined.jsonl'

# Function to process each entry
def process_entry(entry):
    user_parts = []
    model_response = ""
    for message in entry['messages']:
        if message['role'] in ['system', 'user']:
            user_parts.append(message['content'])
        elif message['role'] == 'assistant':
            model_response = message['content']
    user_prompt = " ".join(user_parts)
    formatted_entry = {
        "text": f"<start_of_turn>user {user_prompt} <end_of_turn>\n<start_of_turn>model {model_response} <end_of_turn>",
        "instruction": user_prompt,
        "input": "",
        "output": model_response,
        "prompt": f"<start_of_turn>user {user_prompt} <end_of_turn>\n<start_of_turn>model {model_response} <end_of_turn>"
    }
    return formatted_entry

# Read, process, and write data
with open(input_file_path, 'r', encoding='utf-8') as infile, open(output_file_path, 'w', encoding='utf-8') as outfile:
    for line in infile:
        entry = json.loads(line)
        formatted_entry = process_entry(entry)
        json.dump(formatted_entry, outfile)
        outfile.write('\n')

# Print first two lines to verify
with open(output_file_path, 'r', encoding='utf-8') as processed_file:
    first_line = processed_file.readline()
    second_line = processed_file.readline()
    print(first_line)
    print(second_line)

print("Data processing complete. Formatted data saved to:", output_file_path)


{"text": "<start_of_turn>user Provide answers in Python. Write a program to check if a given string is a valid palindrome or not ignoring case. <end_of_turn>\n<start_of_turn>model def is_palindrome(s):\n    # Convert the string to lowercase\n    s = s.lower()\n    \n    # Remove all non-alphanumeric characters from the string\n    s = ''.join(c for c in s if c.isalnum())\n    \n    # Check if the string is equal to its reverse\n    return s == s[::-1] <end_of_turn>", "instruction": "Provide answers in Python. Write a program to check if a given string is a valid palindrome or not ignoring case.", "input": "", "output": "def is_palindrome(s):\n    # Convert the string to lowercase\n    s = s.lower()\n    \n    # Remove all non-alphanumeric characters from the string\n    s = ''.join(c for c in s if c.isalnum())\n    \n    # Check if the string is equal to its reverse\n    return s == s[::-1]", "prompt": "<start_of_turn>user Provide answers in Python. Write a program to check if a given 

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer

# Assuming the tokenizer has already been loaded
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")

# Load formatted data as a dataset
raw_datasets = load_dataset('json', data_files={'train': output_file_path})

# Shuffle the dataset
shuffled_dataset = raw_datasets['train'].shuffle(seed=1234)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length")

tokenized_datasets = shuffled_dataset.map(tokenize_function, batched=True)

# Split the dataset into training and testing (90% training, 10% testing)
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)

# Properly defining train_data and test_data
train_data = train_test_split['train']
test_data = train_test_split['test']

# Verify the first example from each set
print("Training Data Example:", train_data[0])
print("Testing Data Example:", test_data[0])


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/934 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Training Data Example: {'text': '<start_of_turn>user Provide answers in Python. एक प्रोग्राम लिखें जो एक स्ट्रिंग को इनपुट के रूप में लेता है और स्ट्रिंग का उलटा प्रिंट करता है। <end_of_turn>\n<start_of_turn>model def reverse_string(string):\n    reversed_string = string[::-1]\n    return reversed_string <end_of_turn>', 'instruction': 'Provide answers in Python. एक प्रोग्राम लिखें जो एक स्ट्रिंग को इनपुट के रूप में लेता है और स्ट्रिंग का उलटा प्रिंट करता है।', 'input': '', 'output': 'def reverse_string(string):\n    reversed_string = string[::-1]\n    return reversed_string', 'prompt': '<start_of_turn>user Provide answers in Python. एक प्रोग्राम लिखें जो एक स्ट्रिंग को इनपुट के रूप में लेता है और स्ट्रिंग का उलटा प्रिंट करता है। <end_of_turn>\n<start_of_turn>model def reverse_string(string):\n    reversed_string = string[::-1]\n    return reversed_string <end_of_turn>', 'input_ids': [2, 106, 1645, 34428, 10523, 575, 21237, 235265, 15848, 12218, 28513, 5444, 30860, 113117, 6300, 38750, 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer


from peft import LoraConfig, get_peft_model
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
)

In [ ]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=1,
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    evaluation_strategy='steps',
    eval_steps=112,
    eval_accumulation_steps=1,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
)


In [ ]:
#new code using SFTTrainer
import transformers
from trl import SFTTrainer
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="text",  # Adjust if your dataset has a different text field
    tokenizer=tokenizer,
    max_seq_length=512,  # Adjust based on your data and memory capacity
    args=training_arguments,
    packing=False,
)


Map:   0%|          | 0/840 [00:00<?, ? examples/s]

Map:   0%|          | 0/94 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=105, training_loss=1.4064613580703735, metrics={'train_runtime': 309.9092, 'train_samples_per_second': 2.71, 'train_steps_per_second': 0.339, 'total_flos': 3491098211567616.0, 'train_loss': 1.4064613580703735, 'epoch': 1.0})

In [ ]:
new_model = "gemma-Code-Instruct-Finetune-test" #Name of the model you will be pushing to huggingface model hub
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

In [ ]:
new_model_dir = '/content/drive/MyDrive/ACL/MultilingualLLMBias/tmp_CodeGemma'  # Specify your path in Google Drive
trainer.model.save_pretrained(new_model_dir)


# delete the runtime and re-connnect, because GPU memory is not enough

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install transformers peft huggingface_hub


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from huggingface_hub import login

# 登录 Hugging Face
login()  # 你需要输入你的 Hugging Face 访问令牌


ImportError: cannot import name 'push_to_hub' from 'huggingface_hub' (/usr/local/lib/python3.10/dist-packages/huggingface_hub/__init__.py)

In [ ]:
# 指定模型和 tokenizer 的路径
model_id = "google/codegemma-7b-it"  # 替换为你使用的 base model id
fine_tuned_model_dir = '/content/drive/MyDrive/ACL/MultilingualLLMBias/tmp_CodeGemma'
output_dir = "merged_model"  # 合并后的模型保存路径


# 加载基础模型
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,  # 使用 float16 节省显存
    device_map={"": "cuda:0"}   # 如果使用 GPU，指定 GPU 设备
)

# 加载 LoRA 适配器并合并
merged_model = PeftModel.from_pretrained(base_model, fine_tuned_model_dir)
merged_model = merged_model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository


In [ ]:
# 保存合并后的模型
merged_model.save_pretrained(output_dir, safe_serialization=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.save_pretrained(output_dir)



('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.model',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

In [ ]:
from huggingface_hub import Repository
new_model_name = "Mingda/CodeGemma-Instruct-Finetune-v0.1"  # Ensure the repo name is correct

# Initialize API
api = HfApi()
# Push the model
api.upload_folder(
    repo_id=new_model_name,
    folder_path=output_dir,
    commit_message="Upload fine-tuned model"
)

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mingda/CodeGemma-Instruct-Finetune-v0.1/commit/83b7c96736cada66013b4a26246732549adaabae', commit_message='Upload fine-tuned model', commit_description='', oid='83b7c96736cada66013b4a26246732549adaabae', pr_url=None, pr_revision=None, pr_num=None)